# Proyecto Integrador - Matemáticas Aplicadas a Ciencia de Datos

## Introducción
El presente proyecto integrador tiene como finalidad aplicar conocimientos avanzados de Machine Learning e Inferencia Bayesiana para analizar datos reales y construir modelos predictivos.

El trabajo se fundamenta en el análisis de un archivo histórico proporcionado por una empresa, abarcando el periodo comprendido entre 2019 y 2025. El objetivo central es demostrar el dominio en el ciclo completo de la ciencia de datos: desde la limpieza y transformación de datos crudos hasta la interpretación matemática de los resultados y la justificación teórica del modelo seleccionado para la predicción de la demanda.

## Planteamiento del problema
La organización disponía de un conjunto de datos históricos de pedidos y facturación que originalmente presentaba una estructura de "formato ancho" (wide format), con columnas correspondientes a periodos mensuales, lo cual resultaba ineficiente para el modelado de series de tiempo.

El problema principal radica en la necesidad de transitar de una planeación reactiva a una proactiva para optimizar la cadena de suministro. Dada la volatilidad del mercado y la existencia de patrones estacionales, se requería un sistema capaz de:

1. Capturar ciclos estacionales y tendencias a largo plazo.
2. Enfocar los recursos computacionales en los artículos de mayor impacto comercial, aplicando el principio de Pareto para seleccionar los **Top 10 Productos** y **Top 5 Clientes**.
3. Generar pronósticos robustos con un horizonte de 12 meses para la toma de decisiones de inventario.

## Descripción del Dataset

### Descripción del Dataset y Variables
* **Dataset:** Archivo "Datos Históricos de Pedidos y de Facturación.csv" que cubre el periodo 2019-2025.
* **Estructura Original:** Formato ancho (columnas por meses) transformado a formato largo (melt) para análisis transaccional.

**Variables Principales:**
* **Fecha:** Variable temporal, índice de la serie de tiempo.
* **Producto_Descripcion:** Identificador categórico del ítem vendido.
* **Cliente_Descripcion:** Identificador categórico del comprador.
* **Pedido_Piezas:** Variable Objetivo. Cantidad numérica discreta de demanda.

## Desarrollo teórico

### Formulación matemática del modelo
El modelo seleccionado es un Ensamble por Promedio (Averaging Ensemble) que combina tres predictores bases: Random Forest, SARIMA y Prophet.

**Ecuación del Ensamble:**
La predicción final $\hat{y}_{t}$ para un tiempo $t$ se define como el promedio aritmético de las predicciones individuales:

$$
\hat{y}_{t}=\frac{1}{3}(\hat{y}_{RF}(t)+\hat{y}_{SARIMA}(t)+\hat{y}_{Prophet}(t))
$$

Donde cada componente $f_{k}$ tiene su propia estructura matemática interna:

1.  **$\hat{y}_{RF}(t)$ (Componente No Paramétrico):** Es el promedio de las predicciones de $M$ árboles de decisión de regresión $(h_{m})$, donde la entrada $X$ incluye variables temporales transformadas (año, mes, día).
    $$
    \mathcal{F}_{RF}=\frac{1}{M}\sum_{m=1}^{M}h_{m}(x)
    $$

2.  **$\hat{y}_{SARIMA}(t)$ (Componente Estocástico Lineal):** Modela la serie de tiempo basándose en sus propios retardos y errores pasados, asumiendo estacionariedad tras diferenciación.
    $$
    \Phi_{P}(B)\Phi_{P}(B^{s})(1-B)^{d}(1-B^{s})^{D}y_{t}=\theta_{q}(B)\Theta_{Q}(B^{s})\epsilon_{t}
    $$
    Donde $B$ es el operador de retardo, y $\Phi, \Theta$ son los polinomios de parámetros autorregresivos y de media móvil.

3.  **$\hat{y}_{Prophet}(t)$ (Componente Aditivo Generalizado):** Descompone la serie en tendencia $g(t)$, estacionalidad $s(t)$ (modelada con series de Fourier) y efectos de días festivos $h(t)$.
    $$
    y(t)=g(t)+s(t)+h(t)+\epsilon_{t}
    $$

### Supuestos Matemáticos implícitos
* **Independencia de Errores:** Se asume que los errores de los modelos individuales no están perfectamente correlacionados entre sí. Al promediarlos, la varianza del error total del ensamble tiende a reducirse comparada con la varianza de los modelos individuales, mejorando la robustez.
* **Ponderación Equitativa:** El uso de un promedio simple $(1/3)$ asume que los tres modelos tienen una capacidad predictiva a priori comparable o que se desea evitar el sobreajuste a un solo tipo de algoritmo.

### Función de Costo
Para la evaluación y selección del modelo final en el proyecto, se utilizó la Raíz del Error Cuadrático Medio (RMSE) como métrica de pérdida global.

$$
RMSE=\sqrt{\frac{1}{N}\sum_{i=1}^{N}(y_{i}-\hat{y}_{i})^{2}}
$$

Sin embargo, cada componente matemático optimiza su propia función de costo interna durante el entrenamiento:
* **Random Forest:** Minimiza el Mean Squared Error (MSE) en cada división de los nodos del árbol para reducir la varianza.
* **SARIMA:** Utiliza la Estimación de Máxima Verosimilitud (MLE). Busca los parámetros que maximizan la probabilidad de observar los datos dados: $L(\theta | data)$.

### Método de Optimización
Dado que el modelo final es un promedio simple, no hubo un algoritmo de optimización iterativo para encontrar "pesos" del ensamble (los pesos son fijos a $1/3$). No obstante, los sub-modelos utilizan los siguientes métodos:

* **Algoritmo para SARIMA (Máxima Verosimilitud / Filtro de Kalman):** Conceptualmente, el algoritmo ajusta los coeficientes autorregresivos (AR) y de media móvil (MA) recorriendo el espacio de parámetros hasta encontrar aquellos que hacen que los datos históricos sean "lo más probables posible" bajo la distribución asumida (generalmente Gaussiana). Es un método de optimización basado en gradientes o métodos numéricos (como BFGS) sobre la función de verosimilitud.
* **Algoritmo para Random Forest (Bagging y Partición Recursiva):** No usa descenso del gradiente. Utiliza un enfoque voraz (greedy). En cada nodo, selecciona aleatoriamente un subconjunto de características (en este caso: año, mes, día, festivo) y busca el punto de corte que maximice la reducción de la impureza (varianza) de la variable objetivo (Pedido_Piezas).

**Justificación de funcionamiento:** Este enfoque híbrido funciona porque combina optimización paramétrica (SARIMA/Prophet ajustando curvas suaves) con optimización no paramétrica (Random Forest segmentando el espacio de decisiones), cubriendo diferentes naturalezas del error.

### Propiedades Matemáticas Relevantes
* **Reducción de Varianza (Teorema del Ensamble):** La propiedad fundamental que justifica este modelo es que la varianza del promedio de variables aleatorias independientes (o débilmente correlacionadas) es menor que la varianza de las variables individuales.
    $$
    Var(\overline{x})=\frac{1}{N^{2}}\sum Var(x_{i})+Covarianzas
    $$
    Al promediar modelos distintos (RF, SARIMA, Prophet), se busca reducir el error de varianza sin aumentar significativamente el sesgo.
* **Estacionalidad y Descomposición:** El modelo asume que la serie temporal $Y_{t}$ puede descomponerse en componentes aditivos (Tendencia + Estacionalidad + Residuo). Esto fue validado matemáticamente mediante la prueba de descomposición estacional (seasonal_decompose), confirmando la existencia de patrones cíclicos anuales.
* **Naturaleza No Lineal:** A diferencia de una regresión lineal simple, el espacio de hipótesis de este modelo no está restringido a hiperplanos. Random Forest permite aproximar funciones no lineales complejas y discontinuidades.

### Pros y Contras del Modelo (Perspectiva Matemática)

**Pros:**
* **Robustez Estocástica:** Al no depender de una sola asunción estadística (ej. normalidad de los residuos necesaria para SARIMA), el modelo es más robusto ante violaciones de supuestos.
* **Manejo de Multicolinealidad:** Random Forest (uno de los componentes) es naturalmente inmune a la multicolinealidad entre variables predictoras (como mes y día del año), ya que selecciona características aleatoriamente en cada división.
* **Flexibilidad ante la Estacionalidad:** Puede modelar tanto estacionalidad estricta (SARIMA) como efectos de calendario irregulares (Prophet con festivos) simultáneamente.

**Contras:**
* **Ausencia de Solución Cerrada:** No existe una fórmula analítica única $y=f(x)$ compacta para describir el modelo completo. Es una "caja negra" algorítmica compuesta.
* **Complejidad Computacional:** El costo de entrenamiento $O(T_{RF}+T_{SARIMA}+T_{Prophet})$ triplicando el tiempo de cómputo necesario frente a un modelo simple.
* **Sensibilidad a la No-Estacionariedad en Media:** Si la tendencia futura cambia drásticamente (cambio de régimen estructural fuera del rango histórico 2019-2025), el componente Random Forest no puede extrapolar valores fuera del rango de entrenamiento (a diferencia de la regresión lineal), aunque Prophet intenta compensar esto.

### Argumento Teórico de Adecuación
Este modelo es matemáticamente adecuado para el problema de predicción de demanda de la empresa debido a la estructura intrínseca de los datos detectada:

1.  **Validación de Estructura Cíclica:** El análisis de Feature Importance mostró que dayofyear (día del año) tiene un peso de 0.287, lo que confirma matemáticamente una fuerte dependencia cíclica anual. Los modelos lineales simples fallan aquí, pero la serie de Fourier en Prophet y la diferenciación estacional en SARIMA $(s=12)$ están diseñadas axiomáticamente para resolver esto.
2.  **Mitigación de Sesgo y Varianza:** Los datos reales de ventas suelen tener mucho "ruido" aleatorio.
    * Prophet tiende a tener alto sesgo (sub-ajuste) pero baja varianza (curvas muy suaves).
    * Random Forest tiende a tener bajo sesgo pero alta varianza (sobre-ajuste a picos).
    * El Ensamble por Promedio cancela estos errores opuestos, logrando un compromiso óptimo en el espacio de error, demostrado por el RMSE final de 127,591, que es más estable que los extremos individuales.

## Aplicación práctica del modelo
Para abordar el problema, se implementó una metodología híbrida que combina técnicas supervisadas y no supervisadas en un flujo de trabajo de Ciencia de Datos.

### Descripción de los Datos y Preprocesamiento
Se transformó el dataset a un "formato largo" (melting), consolidando la información en columnas clave: Fecha, Producto_Descripcion, Cliente_Descripcion y la variable objetivo Pedido_Piezas. Se imputaron valores nulos con ceros y se estandarizaron las fechas.

### Implementación del Modelo
El modelo seleccionado es un Meta-modelo de Ensamble que combina tres algoritmos mediante un promedio simple para robustecer la predicción.

**Componentes del Ensamble:**
1.  **Random Forest Regressor:** Configurado con 100 estimadores y profundidad máxima de 8. Su función matemática es capturar relaciones no lineales mediante la reducción de la varianza en los nodos del árbol.
2.  **SARIMA (Seasonal ARIMA):** Configurado con orden (1,1,1) y componente estacional (1,1,1,12). Su función es modelar explícitamente la autocorrelación lineal y la estacionalidad de la serie.
3.  **Prophet:** Incorporado para manejar tendencias cambiantes y el efecto de los días festivos de manera robusta.

### Resultados Principales
El desempeño de los modelos se evaluó utilizando la Raíz del Error Cuadrático Medio (RMSE). El Ensamble demostró ser la opción más equilibrada, mitigando los errores individuales de los modelos base.

**Comparativa de Error (RMSE):**
* SARIMA: 119,770.02
* **Ensemble (Promedio): 127,591.65**
* Random Forest: 148,898.06
* Prophet: 156,091.13

Se generaron pronósticos futuros con intervalos de confianza (Min/Max). Por ejemplo, para el producto "ANALGESICO + ANTIPERETICO TABLETAS 1" y "Cliente 1", el pronóstico para octubre de 2025 es de aproximadamente 380,679 piezas.